In [1]:
# Imports

import plotly
import optuna
import mlflow
import dagshub
import mlflow.sklearn

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [2]:
# Setting up DagsHub

dagshub.init(repo_owner='SushrutGaikwad', repo_name='youtube-comments-analyzer', mlflow=True)

Accessing as SushrutGaikwad

Initialized MLflow to track repo "SushrutGaikwad/youtube-comments-analyzer"

Repository SushrutGaikwad/youtube-comments-analyzer initialized!

# Data

In [3]:
PREPROCESSED_DATA_PATH = "../data/processed/reddit_preprocessed.csv"
df = pd.read_csv(PREPROCESSED_DATA_PATH)
df.dropna(subset=["clean_comment"], inplace=True)
df.shape

(36662, 2)

# Running the experiment

In [4]:
# Setting experiment name

mlflow.set_experiment("Exp 5: Detailed hyperparameter tuning")

<Experiment: artifact_location='mlflow-artifacts:/50f3ef58d8344cd3bd0def333af39f2d', creation_time=1749058038347, experiment_id='6', last_update_time=1749064965442, lifecycle_stage='active', name='Exp 5: Detailed hyperparameter tuning', tags={}>

## Preprocessing

In [5]:
# Remapping class labels from {-1, 0, 1} to {2, 0, 1}
mapping = {
    -1: 2,
    0: 0,
    1: 1
}
df["category"] = df["category"].map(mapping)

# Removing missing sentiments
df.dropna(subset=["category"], inplace=True)

# Feature engineering
ngram_range = (1, 2)
max_features = 1000
vectorizer = CountVectorizer(
    ngram_range=ngram_range,
    max_features=max_features
)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    df["clean_comment"],
    df["category"],
    test_size=0.2,
    random_state=42,
    stratify=df["category"]
)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Undersampling
rus = RandomUnderSampler(random_state=42)
X_train_vectorized, y_train = rus.fit_resample(
    X_train_vectorized,
    y_train
)

## Running the experiment

In [6]:
# Function to log results to MLFlow
def log_to_mlflow(
    model_name,
    model,
    X_train,
    X_test,
    y_train,
    y_test,
    params,
    trial_number
):
    with mlflow.start_run():
        # Tags
        mlflow.set_tag(
            "mlflow.runName", f"Trial_{trial_number}_{model_name}_undersampling_BoW_1000_bigrams"
        )
        mlflow.set_tag("experiment_type", "ML_models_comparison")
        
        # Logging model name as a parameter
        mlflow.log_param("model_name", model_name)
        
        # Logging hyperparameters
        for key, value in params.items():
            mlflow.log_param(key, value)
        
        # Initializing and training the model
        model.fit(X_train, y_train)
        
        # Making predictions on the test set and logging metrics
        y_pred = model.predict(X_test)
        
        # Logging accuracy
        accuracy = accuracy_score(
            y_true=y_test,
            y_pred=y_pred
        )
        mlflow.log_metric("accuracy", accuracy)
        
        # Logging classification report metrics
        classification_rep = classification_report(
            y_true=y_test,
            y_pred=y_pred,
            output_dict=True
        )
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}: {metric}", value)
        
        # Logging the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")
        
        return accuracy

In [7]:
def objective(trial):
    # Hyperparameter space
    n_estimators = trial.suggest_int("n_estimators", 100, 1000)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int("max_depth", 3, 15)
    subsample = trial.suggest_float("subsample", 0.5, 1.0)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.5, 1.0)
    gamma = trial.suggest_float("gamma", 0, 5)
    reg_alpha = trial.suggest_float("reg_alpha", 1e-4, 10.0, log=True)
    reg_lambda = trial.suggest_float("reg_lambda", 1e-4, 10.0, log=True)
    
    params = {
        "n_estimators": n_estimators,
        "learning_rate": learning_rate,
        "max_depth": max_depth,
        "subsample": subsample,
        "colsample_bytree": colsample_bytree,
        "gamma": gamma,
        "reg_alpha": reg_alpha,
        "reg_lambda": reg_lambda
    }
    
    model = XGBClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        gamma=gamma,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        random_state=42,
        n_jobs=-1
    )
    
    # Logging each trial as a separate run
    accuracy = log_to_mlflow(
        model_name="XGBoost",
        model=model,
        X_train=X_train_vectorized,
        X_test=X_test_vectorized,
        y_train=y_train,
        y_test=y_test,
        params=params,
        trial_number=trial.number
    )
    
    return accuracy

In [8]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

[I 2025-06-05 00:53:58,660] A new study created in memory with name: no-name-6ad965c3-8317-4bcd-8bc8-f0aedbd1c03b
2025/06/05 00:54:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Trial_0_XGBoost_undersampling_BoW_1000_bigrams at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/6/runs/38d3fbc574b84130b4b0da821364cf4b
🧪 View experiment at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/6


[I 2025-06-05 00:54:32,497] Trial 0 finished with value: 0.584072003272876 and parameters: {'n_estimators': 912, 'learning_rate': 0.00012551519198700936, 'max_depth': 6, 'subsample': 0.526167618143076, 'colsample_bytree': 0.9466050627637463, 'gamma': 1.216484884032576, 'reg_alpha': 0.010980997223880951, 'reg_lambda': 5.742867085334908}. Best is trial 0 with value: 0.584072003272876.
2025/06/05 00:55:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Trial_1_XGBoost_undersampling_BoW_1000_bigrams at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/6/runs/266f17a0b9e1402a9afb7f0c4d0a38ca
🧪 View experiment at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/6


[I 2025-06-05 00:55:12,170] Trial 1 finished with value: 0.6702577389881358 and parameters: {'n_estimators': 252, 'learning_rate': 0.004590196515363164, 'max_depth': 10, 'subsample': 0.9619760293006714, 'colsample_bytree': 0.6807266982550868, 'gamma': 0.4812308512110547, 'reg_alpha': 0.014268011523763521, 'reg_lambda': 0.12185371052100098}. Best is trial 1 with value: 0.6702577389881358.
2025/06/05 00:55:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

[I 2025-06-05 02:02:45,703] Trial 97 finished with value: 0.7691258693576981 and parameters: {'n_estimators': 647, 'learning_rate': 0.034859133478872505, 'max_depth': 7, 'subsample': 0.8410683427362102, 'colsample_bytree': 0.9766171111958002, 'gamma': 1.082000864874538, 'reg_alpha': 1.4546299452994216, 'reg_lambda': 0.06804941605933168}. Best is trial 75 with value: 0.787126687576708.
2025/06/05 02:03:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Trial_98_XGBoost_undersampling_BoW_1000_bigrams at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/6/runs/ee0300fbc1f44c4fa45b4aaa52dbafe1
🧪 View experiment at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/6


[I 2025-06-05 02:03:27,702] Trial 98 finished with value: 0.7849447702168281 and parameters: {'n_estimators': 710, 'learning_rate': 0.08331554529903365, 'max_depth': 8, 'subsample': 0.7243044999330589, 'colsample_bytree': 0.9095789736408462, 'gamma': 1.3011982672827336, 'reg_alpha': 0.1154116520992475, 'reg_lambda': 0.012742827140762063}. Best is trial 75 with value: 0.787126687576708.
2025/06/05 02:04:04 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Trial_99_XGBoost_undersampling_BoW_1000_bigrams at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/6/runs/64ebe46dbc6d456eb91f84eda936e62f
🧪 View experiment at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/6


[I 2025-06-05 02:04:09,905] Trial 99 finished with value: 0.6551206873039683 and parameters: {'n_estimators': 503, 'learning_rate': 0.004797578432783995, 'max_depth': 6, 'subsample': 0.7740079986903426, 'colsample_bytree': 0.8704615980481, 'gamma': 1.7143546167128956, 'reg_alpha': 0.2302147964350328, 'reg_lambda': 0.09580747960066929}. Best is trial 75 with value: 0.787126687576708.


In [9]:
# Getting the best parameters
best_params = study.best_params
best_model = XGBClassifier(
    n_estimators=best_params["n_estimators"],
    learning_rate=best_params["learning_rate"],
    max_depth=best_params["max_depth"],
    subsample=best_params["subsample"],
    colsample_bytree=best_params["colsample_bytree"],
    gamma=best_params["gamma"],
    reg_alpha=best_params["reg_alpha"],
    reg_lambda=best_params["reg_lambda"],
    random_state=42,
    n_jobs=-1
)

In [10]:
# Logging the best model
log_to_mlflow(
    model_name="XGBoost",
    model=best_model,
    X_train=X_train_vectorized,
    X_test=X_test_vectorized,
    y_train=y_train,
    y_test=y_test,
    params=best_params,
    trial_number="best"
)

2025/06/05 02:05:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Trial_best_XGBoost_undersampling_BoW_1000_bigrams at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/6/runs/c4cf4df1b728423caa99940187c98509
🧪 View experiment at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/6


0.787126687576708

In [11]:
# Plotting parameter importance
optuna.visualization.plot_param_importances(study).show()

# Plotting optimization history
optuna.visualization.plot_optimization_history(study).show()